# Problem klasyfikacji gęstości (DCP)

### Imports

In [1]:
# pip install joblib

In [2]:
import math
import numpy as np
from operator import itemgetter
from joblib import Parallel, delayed

Funkcje generujące diagram czasoprzestrzenny:

In [3]:
def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)], dtype=np.uint8)

def eca_evolve(lut: np.ndarray, x: np.ndarray) -> np.ndarray:
    return lut[7 - (np.roll(x, 1) * 4 + x * 2 + np.roll(x, -1))]

def eca_evolve_spacetime(lut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(eca_evolve(lut, rows[-1]))
    return np.stack(rows)


Funkcja do losowania konfiguracji początkowej:

In [4]:
def generate_random_initial_configuration(length: int) -> np.ndarray:
    return np.random.randint(2, size=length)

## Wczytywanie konfiguracji pana Marcina
Pliki z konfiguracjami można pobrać z https://github.com/D3M80L/CA/tree/master/phd/data.

Funkcja konwertująca ciąg bajtów na binarną reprezentację. Zwraca tablicę N pierwszych bitów:

In [5]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


Funkcja wczytująca wszystkie konfiguracje z pliku do pamięci operacyjnej:

In [6]:
def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = math.ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)
            
            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)


- Pierwszy argument (str): nazwa pliku, zktórego chcemy pobrać konfiguracje.
- Drugi argument (int): długość konfiguracji zapisanych w pliku.
- Trzeci argument (bool): czy w wynikowej liście mają znaleźć się też "zanegowane" konfiguracje; False - konfiguracje będą miały gęstość mniejszą niż 0.5; True - konfiguracje, których gęstość jest mniejsza jaki i większa od 0.5.

Przykład użycia:

In [7]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
configurations2 = configuration_reader("ALL_N21.bin", 21, False)

In [8]:
print(configurations[0:10])
print(configurations2[0:10])

[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1]]


---

# zad.7
Dla ustalonego N, przebadaj wszystkie ECAs dając im limit czasu i wylicz dla jakiego % wszystkich konfiguracji początkowych otrzymaną poprawną klasyfikację gęstości. Zaprezentuj wyniki w postaci listy par (numer reguły, % poprawnych klasyfikacji), np.

(0, 50%)\
...\
(255, 50%)

Znajdź / wyróżnij regułę, która ma największy wśród ECAs % poprawnych klasyfikacji.

In [ ]:
# gęstość konfiguracji po N krokach porównujemy z gęstością konfiguracji początkowej
# konfiguracje początkowe do badania weźmiemy z pliku ALL_N21.bin
# limit czasu to maksymalna długość kroków w diagramie czasoprzestrzennym (umownie 2x długość konfiguracji początkowej)

In [9]:
def dcp(lut, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    config = eca_evolve_spacetime(lut, initial_config, steps)[-1]
    result_density = sum(config)/len(config)

    if initial_density < 0.5:
        if result_density < 0.5:
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if result_density > 0.5:
            return 1
        else:
            return 0
    return 0


In [10]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
len(configurations)

99880

In [14]:
results = []
for rule in range(256):
    print(f'Rule {rule}...')
    lut = eca_get_lut(rule)
    # l = [dcp(lut, config, 42) for config in configurations]
    l = Parallel(n_jobs=-1)(delayed(dcp)(lut, config, 42) for config in configurations)
    results.append((rule, round(100*sum(l)/len(configurations),3)))
    print(f'({results[-1][0]}, {results[-1][1]}%)')

Rule 0...
(0, 50.0%)
Rule 1...
(1, 49.593%)
Rule 2...
(2, 50.0%)
Rule 3...
(3, 46.942%)
Rule 4...
(4, 50.0%)
Rule 5...
(5, 46.942%)
Rule 6...
(6, 50.0%)
Rule 7...
(7, 23.812%)
Rule 8...
(8, 50.0%)
Rule 9...
(9, 50.376%)
Rule 10...
(10, 50.0%)
Rule 11...
(11, 31.478%)
Rule 12...
(12, 50.0%)
Rule 13...
(13, 49.999%)
Rule 14...
(14, 42.131%)
Rule 15...
(15, 0.0%)
Rule 16...
(16, 50.0%)
Rule 17...
(17, 46.942%)
Rule 18...
(18, 50.0%)
Rule 19...
(19, 22.297%)
Rule 20...
(20, 50.0%)
Rule 21...
(21, 23.812%)
Rule 22...
(22, 49.796%)
Rule 23...
(23, 10.37%)
Rule 24...
(24, 50.0%)
Rule 25...
(25, 51.985%)
Rule 26...
(26, 50.588%)
Rule 27...
(27, 23.357%)
Rule 28...
(28, 36.925%)
Rule 29...
(29, 0.0%)
Rule 30...
(30, 49.98%)
Rule 31...
(31, 23.812%)
Rule 32...
(32, 50.0%)
Rule 33...
(33, 42.122%)
Rule 34...
(34, 50.0%)
Rule 35...
(35, 45.157%)
Rule 36...
(36, 50.0%)
Rule 37...
(37, 51.356%)
Rule 38...
(38, 49.808%)
Rule 39...
(39, 23.357%)
Rule 40...
(40, 50.019%)
Rule 41...
(41, 50.06%)
Rule 42

In [16]:
sorted(results, key=itemgetter(1))

[(15, 0.0),
 (29, 0.0),
 (51, 0.0),
 (71, 0.0),
 (85, 0.0),
 (23, 10.37),
 (19, 22.297),
 (55, 22.297),
 (27, 23.357),
 (39, 23.357),
 (53, 23.357),
 (83, 23.357),
 (7, 23.812),
 (21, 23.812),
 (31, 23.812),
 (87, 23.812),
 (11, 31.478),
 (47, 31.478),
 (81, 31.478),
 (117, 31.478),
 (28, 36.925),
 (70, 36.925),
 (157, 36.925),
 (199, 36.925),
 (156, 37.889),
 (198, 37.889),
 (43, 40.527),
 (113, 40.527),
 (33, 42.122),
 (123, 42.122),
 (14, 42.131),
 (84, 42.131),
 (143, 42.131),
 (213, 42.131),
 (35, 45.157),
 (49, 45.157),
 (59, 45.157),
 (115, 45.157),
 (77, 45.985),
 (3, 46.942),
 (5, 46.942),
 (17, 46.942),
 (63, 46.942),
 (95, 46.942),
 (119, 46.942),
 (105, 48.57),
 (122, 49.187),
 (161, 49.187),
 (73, 49.366),
 (109, 49.366),
 (1, 49.593),
 (127, 49.593),
 (62, 49.595),
 (118, 49.595),
 (131, 49.595),
 (145, 49.595),
 (22, 49.796),
 (151, 49.796),
 (38, 49.808),
 (52, 49.808),
 (155, 49.808),
 (211, 49.808),
 (110, 49.864),
 (124, 49.864),
 (137, 49.864),
 (193, 49.864),
 (54,

In [17]:
sorted(results, key=itemgetter(1), reverse=True)[0:5]

[(170, 100.0), (184, 100.0), (204, 100.0), (226, 100.0), (240, 100.0)]

Z badania otrzymaliśmy pięć reguł, które mają 100% poprawnych klasyfikacji gęstości dla konfiguracji o długości 21:
- 170 (left shift),
- 184 (traffic right),
- 204 (identity),
- 226 (trafic left),
- 240 (right shift).

# zad.8
Henryk Fukś pokazał, że idealne rozwiązanie DCP można uzyskać stosując ECA 184 przez "połowę czasu", a następnie ECA 232 przez następną 
połowę czasu.

Stosując wybrany "zbiór Marcina" lub zbiór wygenerowany losowo sprawdź, czy odkrycie prof. Fuksia daje się odtworzyć eksperymentalnie.

**Wariant (*)** - czy poza ECA 184 istnieją inne ECAs, które w powiązaniu z ECA 232 dają taką własność? Chodzi o inną parę ECA x, ECA 232 lub ewentualnie ECA x, ECA y, które stosowane tak jak przez prof. Fuksia rozwiązują DCP dokładnie.

## 184 - traffic right and 232 - majority

In [11]:
def dcp2(lut1, lut2, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    config = eca_evolve_spacetime(lut1, initial_config, steps//2)[-1]
    result_config = eca_evolve_spacetime(lut2, config, steps//2)[-1]
    result_density = sum(result_config)/len(result_config)

    if initial_density < 0.5:
        if result_density < 0.5:
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if result_density > 0.5:
            return 1
        else:
            return 0
    return 0


In [12]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut184 = eca_get_lut(184)
lut232 = eca_get_lut(232)
# l = [dcp2(lut184, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut184, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 100.0%


## 226 - traffic left and 232 - majority

In [13]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut226 = eca_get_lut(226)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut226, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 100.0%


## 170 - left shift and 232 - majority

In [18]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut170 = eca_get_lut(170)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut170, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


## 204 - identity and 232 - majority

In [19]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut204 = eca_get_lut(204)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut204, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


## 240 - right shift and 232 - majority

In [20]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut240 = eca_get_lut(240)
lut232 = eca_get_lut(232)
# l = [dcp2(lut226, lut232, config, 42) for config in configurations]
l = Parallel(n_jobs=-1)(delayed(dcp2)(lut240, lut232, config, 42) for config in configurations)
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),3)}%')

Poprawna klasyfikacja gęstości: 89.63%


# zad.9 (zmodyfikowane - patrz lab5.ipynb)
<s>Zapoznaj się z wynikami z artykułu: *Mendonça, J. R. G. (2019).* **Simply modified GKL density classifiers that reach consensus faster.** *Physics Letters A, 383(19), 2264–2266.* doi:10.1016/j.physleta.2019.04.033 (https://scihub.se/https://www.sciencedirect.com/science/article/abs/pii/S0375960119303512?via=ihub).</s>

<s>Spróbuj powtórzyć eksperymenty tam wykonane dla wybranego N dla którego mamy "zbiór Marcina" badając średni czas klasyfikacji oraz % poprawnych klasyfikacji dla reguł GKL(j, k) dla kilku przypadków j, k.</s>

<s>Otrzymane wyniki porównaj z tymi z Zadania 7. To znaczy jak bardzo lepsze są najlepsze reguły GKL, które uda Ci się znaleźć w porównaniu do najlepszych ECAs.</s>

# <del>zad.10(*)</del>
<del>Sprawdź jak wybór zbioru testowych warunków początkowych wypływa na wyniki eksperymentów z Zadania 8.</del>

<del>Czy jesteś w stanie znaleźć przykład parametrów N (liczba komórek), num_conf (liczba losowanych warunków w zbiorze), p (prawdopodobieństwo 0 w 
komórce) do losowania, aby pokazać, że eliminacja duplikatów (shift-equal) ma znaczenie - tj. niechlujne dobranie "losowego" zbioru warunków początkowych zawyża / zaniża szacowaną jakość klasyfikacji.</del>

<del>**Podpowiedź**: Możesz porównać wyniki otrzymane na wybranym "zbiorze Marcina" z wynikami na zbiorze otrzymanym w zwykłym losowaniu dla konkretnego N.</del>